# Coronavirus v2

## Goals

Determine the number of people served by every hospital in the continental US.

## Strategy
1. Map each zip code to the hospital which serves it
    1. Create the Voronoi diagram of hospitals
    2. If a zip code is in the voronoi diagram, map hospital to zip code and zip code to hospital
    3. https://geopandas.org/mergingdata.html#sjoin-arguments
2. Sum up the number of people in each zip code served by each hospital.

## Notes
On SageMaker, you'll have to `conda install rtree -n python3`

In [1]:
import pandas as pd
import geopandas as gpd
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from shapely.geometry import LineString
from shapely.ops import polygonize


In [5]:
hospital = pd.read_csv('./Hospitals.csv')


# what is the key info for the hospital dataset? Just the coordinates and the number of beds.
hospital_cols = [
    'X', 'Y', 'ID', 'NAME', 'ADDRESS', 
    'CITY', 'STATE', 'ZIP', 'TYPE', 
    'POPULATION', 'COUNTY', 
    'LATITUDE', 'LONGITUDE',
    'WEBSITE',
    'TTL_STAFF', 'BEDS', 'TRAUMA', 'HELIPAD'
]

# filter out the hospitals that aren't open and don't have bed data
h_df = hospital[hospital_cols][(hospital['STATUS'] != 'CLOSED').multiply(hospital['BEDS'] != -999)]

# create the geopandas dataframe
h_points = [Point(xy) for xy in zip(h_df['LONGITUDE'], h_df['LATITUDE'])]
h_gdf = gpd.GeoDataFrame(h_df, geometry = h_points)

# create the voronoi diagram for hospitals
h_points = h_gdf[['LONGITUDE', 'LATITUDE']].values
h_vor = Voronoi(h_points)


# associate the voronoi polygons with each hospital
h_lines = [
    LineString(h_vor.vertices[line])
    for line in h_vor.ridge_vertices
    if -1 not in line
]

h_polygons = [p for p in polygonize(h_lines)]

h_polygons_gdf = gpd.GeoDataFrame(h_polygons, columns = ['polygon'])
h_polygons_gdf.geometry = h_polygons_gdf['polygon']

# merge in the polygons
h2_gdf = gpd.sjoin(h_gdf, h_polygons_gdf, how='left', op='within')

# set the geometry of the gdf to be the polygon instead of the point
h2_gdf.geometry = h2_gdf['polygon']
h2_gdf = h2_gdf.drop('index_right', axis=1) # technical, need to remove col with this name
h2_gdf = h2_gdf.dropna(subset=['polygon'])
len(h_df) # before 

In [6]:
len(h2_gdf) # after the voronoi, some hospitals are dropped

6569

In [7]:
h2_gdf.head()

,X,Y,ID,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,POPULATION,COUNTY,LATITUDE,LONGITUDE,WEBSITE,TTL_STAFF,BEDS,TRAUMA,HELIPAD,geometry,polygon
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,VENTURA,34.154939,-118.815736,http://www.losrobleshospital.com,-999,62,NOT AVAILABLE,N,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...
2,-118.184165,34.023647,11190023,EAST LOS ANGELES DOCTORS HOSPITAL,4060 WHITTIER BOULEVARD,LOS ANGELES,CA,90023,GENERAL ACUTE CARE,127,LOS ANGELES,34.023647,-118.184165,http://www.elalax.com,-999,127,NOT AVAILABLE,N,"POLYGON ((-118.14447 33.99918, -118.19974 34.0...","POLYGON ((-118.1444689431891 33.999176651987, ..."
3,-118.325235,34.096391,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,6245 DE LONGPRE AVENUE,HOLLYWOOD,CA,90028,GENERAL ACUTE CARE,100,LOS ANGELES,34.096391,-118.325235,http://sch-hollywood.com/,-999,100,NOT AVAILABLE,N,"POLYGON ((-118.31182 34.12763, -118.30836 34.0...",POLYGON ((-118.3118233068845 34.12762993533133...
4,-117.967438,34.063039,23691706,KINDRED HOSPITAL BALDWIN PARK,14148 FRANCISQUITO AVENUE,BALDWIN PARK,CA,91706,GENERAL ACUTE CARE,95,LOS ANGELES,34.063039,-117.967438,http://www.khbaldwinpark.com,-999,95,NOT AVAILABLE,N,"POLYGON ((-117.96702 34.09655, -117.95665 34.0...",POLYGON ((-117.9670173107889 34.09654606212992...
5,-118.148403,33.859707,25190712,LAKEWOOD REGIONAL MEDICAL CENTER,3700 EAST SOUTH STREET,LAKEWOOD,CA,90712,GENERAL ACUTE CARE,172,LOS ANGELES,33.859707,-118.148403,http://www.lakewoodregional.com,-999,172,NOT AVAILABLE,N,"POLYGON ((-118.12219 33.82519, -118.15287 33.8...",POLYGON ((-118.1221887350056 33.82519025963015...


In [8]:
# now get the zip code data
z_df = pd.read_csv('./Geocodes_USA_with_Counties.csv')
z_columns = ['zip', 'primary_city', 'state', 
             'latitude', 'longitude', 'county',
             'estimated_population']
z_df = z_df[z_df['decommissioned'] != 1][z_columns]

# create the geopandas data frame
z_points = [Point(xy) for xy in zip(z_df['longitude'], z_df['latitude'])]
z_gdf = gpd.GeoDataFrame(z_df, geometry = z_points)
z_gdf.head()

,zip,primary_city,state,latitude,longitude,county,estimated_population,geometry
0,501,Holtsville,NY,40.81,-73.04,Suffolk,384,POINT (-73.04000 40.81000)
1,544,Holtsville,NY,40.81,-73.04,Suffolk,0,POINT (-73.04000 40.81000)
2,601,Adjuntas,PR,18.16,-66.72,Adjuntas,0,POINT (-66.72000 18.16000)
3,602,Aguada,PR,18.38,-67.18,NaN,0,POINT (-67.18000 18.38000)
4,603,Aguadilla,PR,18.43,-67.15,Aguadilla,0,POINT (-67.15000 18.43000)


In [9]:
# merge the hospital and the zip_codes
m_gdf = gpd.sjoin(h2_gdf, z_gdf, how='left', op='contains')

In [10]:
m_gdf.head()

,X,Y,ID,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,POPULATION,...,geometry,polygon,index_right,zip,primary_city,state,latitude,longitude,county,estimated_population
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,38452.0,91377.0,Oak Park,CA,34.19,-118.76,Ventura,12293.0
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,38444.0,91362.0,Thousand Oaks,CA,34.19,-118.81,Ventura,31759.0
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,38441.0,91359.0,Westlake Village,CA,34.19,-118.82,Los Angeles,1244.0
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,38451.0,91376.0,Agoura Hills,CA,34.14,-118.75,Los Angeles,642.0
1,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,38396.0,91301.0,Agoura Hills,CA,34.12,-118.76,Los Angeles,22555.0


In [11]:
print(len(z_df))
print(len(m_gdf))

41859
42978


In [13]:
m_hcount = {}

for h_name in m_gdf['ID']:
    m_hcount[h_name] = m_gdf[m_gdf['ID'] == h_name]['estimated_population'].sum()

In [14]:
hcount_df = pd.DataFrame(
    [
        dict(zip(list(range(len(m_hcount))), m_hcount.keys())), 
        dict(zip(list(range(len(m_hcount))), m_hcount.values()))
    ]
).transpose()

hcount_df.columns = ['ID', 'total_population']
hcount_df.head()

,ID,total_population
0,53391362.0,81982.0
1,11190023.0,35693.0
2,17090028.0,39050.0
3,23691706.0,701.0
4,25190712.0,27271.0


In [15]:
h3_gdf = pd.merge(h2_gdf, hcount_df, on='ID', how='outer')
h3_gdf.head()

,X,Y,ID,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,POPULATION,...,LATITUDE,LONGITUDE,WEBSITE,TTL_STAFF,BEDS,TRAUMA,HELIPAD,geometry,polygon,total_population
0,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,62,...,34.154939,-118.815736,http://www.losrobleshospital.com,-999,62,NOT AVAILABLE,N,"POLYGON ((-118.72682 34.19377, -118.72198 33.9...",POLYGON ((-118.7268193170333 34.19377013146838...,81982.0
1,-118.184165,34.023647,11190023,EAST LOS ANGELES DOCTORS HOSPITAL,4060 WHITTIER BOULEVARD,LOS ANGELES,CA,90023,GENERAL ACUTE CARE,127,...,34.023647,-118.184165,http://www.elalax.com,-999,127,NOT AVAILABLE,N,"POLYGON ((-118.14447 33.99918, -118.19974 34.0...","POLYGON ((-118.1444689431891 33.999176651987, ...",35693.0
2,-118.325235,34.096391,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,6245 DE LONGPRE AVENUE,HOLLYWOOD,CA,90028,GENERAL ACUTE CARE,100,...,34.096391,-118.325235,http://sch-hollywood.com/,-999,100,NOT AVAILABLE,N,"POLYGON ((-118.31182 34.12763, -118.30836 34.0...",POLYGON ((-118.3118233068845 34.12762993533133...,39050.0
3,-117.967438,34.063039,23691706,KINDRED HOSPITAL BALDWIN PARK,14148 FRANCISQUITO AVENUE,BALDWIN PARK,CA,91706,GENERAL ACUTE CARE,95,...,34.063039,-117.967438,http://www.khbaldwinpark.com,-999,95,NOT AVAILABLE,N,"POLYGON ((-117.96702 34.09655, -117.95665 34.0...",POLYGON ((-117.9670173107889 34.09654606212992...,701.0
4,-118.148403,33.859707,25190712,LAKEWOOD REGIONAL MEDICAL CENTER,3700 EAST SOUTH STREET,LAKEWOOD,CA,90712,GENERAL ACUTE CARE,172,...,33.859707,-118.148403,http://www.lakewoodregional.com,-999,172,NOT AVAILABLE,N,"POLYGON ((-118.12219 33.82519, -118.15287 33.8...",POLYGON ((-118.1221887350056 33.82519025963015...,27271.0


In [175]:
h3_gdf.head()

,Unnamed: 0,X,Y,ID,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,...,LATITUDE,LONGITUDE,WEBSITE,TTL_STAFF,BEDS,TRAUMA,HELIPAD,geometry,polygon,total_population
0,0,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,...,34.154939,-118.815736,http://www.losrobleshospital.com,-999,62,NOT AVAILABLE,N,POLYGON ((-118.7268193170333 34.19377013146838...,POLYGON ((-118.7268193170333 34.19377013146838...,81982.0
1,1,-118.184165,34.023647,11190023,EAST LOS ANGELES DOCTORS HOSPITAL,4060 WHITTIER BOULEVARD,LOS ANGELES,CA,90023,GENERAL ACUTE CARE,...,34.023647,-118.184165,http://www.elalax.com,-999,127,NOT AVAILABLE,N,"POLYGON ((-118.1444689431891 33.999176651987, ...","POLYGON ((-118.1444689431891 33.999176651987, ...",35693.0
2,2,-118.325235,34.096391,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,6245 DE LONGPRE AVENUE,HOLLYWOOD,CA,90028,GENERAL ACUTE CARE,...,34.096391,-118.325235,http://sch-hollywood.com/,-999,100,NOT AVAILABLE,N,POLYGON ((-118.3118233068845 34.12762993533133...,POLYGON ((-118.3118233068845 34.12762993533133...,39050.0
3,3,-117.967438,34.063039,23691706,KINDRED HOSPITAL BALDWIN PARK,14148 FRANCISQUITO AVENUE,BALDWIN PARK,CA,91706,GENERAL ACUTE CARE,...,34.063039,-117.967438,http://www.khbaldwinpark.com,-999,95,NOT AVAILABLE,N,POLYGON ((-117.9670173107889 34.09654606212992...,POLYGON ((-117.9670173107889 34.09654606212992...,701.0
4,4,-118.148403,33.859707,25190712,LAKEWOOD REGIONAL MEDICAL CENTER,3700 EAST SOUTH STREET,LAKEWOOD,CA,90712,GENERAL ACUTE CARE,...,33.859707,-118.148403,http://www.lakewoodregional.com,-999,172,NOT AVAILABLE,N,POLYGON ((-118.1221887350056 33.82519025963015...,POLYGON ((-118.1221887350056 33.82519025963015...,27271.0


## Analysis

In [17]:
# merely load the csv since the heavy work is done
h3_gdf = pd.read_csv('./merged_data.csv')
h3_gdf.head()

,Unnamed: 0,X,Y,ID,NAME,ADDRESS,CITY,STATE,ZIP,TYPE,...,LATITUDE,LONGITUDE,WEBSITE,TTL_STAFF,BEDS,TRAUMA,HELIPAD,geometry,polygon,total_population
0,0,-118.815736,34.154939,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,GENERAL ACUTE CARE,...,34.154939,-118.815736,http://www.losrobleshospital.com,-999,62,NOT AVAILABLE,N,POLYGON ((-118.7268193170333 34.19377013146838...,POLYGON ((-118.7268193170333 34.19377013146838...,81982.0
1,1,-118.184165,34.023647,11190023,EAST LOS ANGELES DOCTORS HOSPITAL,4060 WHITTIER BOULEVARD,LOS ANGELES,CA,90023,GENERAL ACUTE CARE,...,34.023647,-118.184165,http://www.elalax.com,-999,127,NOT AVAILABLE,N,"POLYGON ((-118.1444689431891 33.999176651987, ...","POLYGON ((-118.1444689431891 33.999176651987, ...",35693.0
2,2,-118.325235,34.096391,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,6245 DE LONGPRE AVENUE,HOLLYWOOD,CA,90028,GENERAL ACUTE CARE,...,34.096391,-118.325235,http://sch-hollywood.com/,-999,100,NOT AVAILABLE,N,POLYGON ((-118.3118233068845 34.12762993533133...,POLYGON ((-118.3118233068845 34.12762993533133...,39050.0
3,3,-117.967438,34.063039,23691706,KINDRED HOSPITAL BALDWIN PARK,14148 FRANCISQUITO AVENUE,BALDWIN PARK,CA,91706,GENERAL ACUTE CARE,...,34.063039,-117.967438,http://www.khbaldwinpark.com,-999,95,NOT AVAILABLE,N,POLYGON ((-117.9670173107889 34.09654606212992...,POLYGON ((-117.9670173107889 34.09654606212992...,701.0
4,4,-118.148403,33.859707,25190712,LAKEWOOD REGIONAL MEDICAL CENTER,3700 EAST SOUTH STREET,LAKEWOOD,CA,90712,GENERAL ACUTE CARE,...,33.859707,-118.148403,http://www.lakewoodregional.com,-999,172,NOT AVAILABLE,N,POLYGON ((-118.1221887350056 33.82519025963015...,POLYGON ((-118.1221887350056 33.82519025963015...,27271.0


### Coronavirus hyperparameters
The 10x rate is about 16 days, so the daily spread is about 1.155x. About 20% of people who get the disease are hospitalized for betwen 2-6 weeks. Case fatality rate is 3%. 

#### How do we model it spreading from zone to zone?

The probability that a new case appears in region $a$ at time $t$ is proportional to the probability that
- a person from region $b$ travels to $a$, which we'll assume is related to the number of people in $a$ and how far it is, or $\frac{n_a}{d(a,b)}$. We'll assume an exponential distribution related to this value, with some hyperparameter $\lambda$.  

- that person communicates the disease to a new person. We know $r_0 = 2.2$. Let's assume that the average case lasts 14 days before someone is hospitalized or recovered, so they have 14 days to infect 2.2 people, thus $E[\text{infections after 14 days}] = 2.2$. If we also assume that the probability of infecting someone on each day is independent, then $E[\text{infections after 14 days}] = 14 * E[\text{infections after one day}]$, so $E[\text{infections after one day}] =  \frac{2.2}{14}$. Finally, since the number of infections after one day is simply an indicator variable, $p(\text{infect someone in one day}) = \frac{2.2}{14} = .157$.

We sum over all of the regions (which we're simplifying to as hospital coverage zones) and over all the cases in region $b$, thus

$$
p(r_a, t) = \sum_{b \in B} \sum_{c_b}  p(\text{communicate}).
$$

The second sum is equivalent to a Binomial distribution with $n = c_b$ and $p = \frac{n_a}{d(a,b)} p(\text{communicate})$.

In [18]:
import math

def coord_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[0] - p1[0])**2)

In [19]:
p_columns = [
    'X', 'Y', 'NAME', 'CITY', 
    'STATE', 'POPULATION', 'TTL_STAFF', 
    'BEDS', 'geometry', 'polygon',
    'total_population'
]

p_df = h3_gdf[p_columns]
p_df['asymptomatic_cases'] = 0 # lasts around 5 days
p_df['critical_cases'] = 0 # can last up to 6-8 weeks, 5% cases
p_df['severe_cases'] = 0 # lasts up to 3-6 weeks, 14% cases
p_df['mild_cases'] = 0 # lasts around 14 days, 81% cases
p_df['deaths'] = 0
p_df['location'] = p_df["X"].astype(str) + "," + p_df["Y"].astype(str)

p_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,X,Y,NAME,CITY,STATE,POPULATION,TTL_STAFF,BEDS,geometry,polygon,total_population,asymptomatic_cases,critical_cases,severe_cases,mild_cases,deaths,location
0,-118.815736,34.154939,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,WESTLAKE VILAGE,CA,62,-999,62,POLYGON ((-118.7268193170333 34.19377013146838...,POLYGON ((-118.7268193170333 34.19377013146838...,81982.0,0,0,0,0,0,"-118.815736391,34.15493887200005"
1,-118.184165,34.023647,EAST LOS ANGELES DOCTORS HOSPITAL,LOS ANGELES,CA,127,-999,127,"POLYGON ((-118.1444689431891 33.999176651987, ...","POLYGON ((-118.1444689431891 33.999176651987, ...",35693.0,0,0,0,0,0,"-118.18416480499997,34.02364730200003"
2,-118.325235,34.096391,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,HOLLYWOOD,CA,100,-999,100,POLYGON ((-118.3118233068845 34.12762993533133...,POLYGON ((-118.3118233068845 34.12762993533133...,39050.0,0,0,0,0,0,"-118.32523487099995,34.096391357000066"
3,-117.967438,34.063039,KINDRED HOSPITAL BALDWIN PARK,BALDWIN PARK,CA,95,-999,95,POLYGON ((-117.9670173107889 34.09654606212992...,POLYGON ((-117.9670173107889 34.09654606212992...,701.0,0,0,0,0,0,"-117.967437788,34.06303893200003"
4,-118.148403,33.859707,LAKEWOOD REGIONAL MEDICAL CENTER,LAKEWOOD,CA,172,-999,172,POLYGON ((-118.1221887350056 33.82519025963015...,POLYGON ((-118.1221887350056 33.82519025963015...,27271.0,0,0,0,0,0,"-118.14840296499996,33.859706620000054"


In [31]:
p_df.sort_values(by=['total_population'], ascending=False).head()

,X,Y,NAME,CITY,STATE,POPULATION,TTL_STAFF,BEDS,geometry,polygon,total_population,asymptomatic_cases,critical_cases,severe_cases,mild_cases,deaths,location
2062,-95.366148,29.748116,ST. JOSEPH MEDICAL CENTER,HOUSTON,TX,744,-999,744,POLYGON ((-95.35688749103775 29.79522250820279...,POLYGON ((-95.35688749103775 29.79522250820279...,2137434.0,0,0,0,0,0,"-95.36614813899996,29.74811612700005"
3921,-87.698277,41.855215,SAINT ANTHONY HOSPITAL,CHICAGO,IL,151,-999,151,POLYGON ((-87.69819343181665 41.85933941974636...,POLYGON ((-87.69819343181665 41.85933941974636...,1955130.0,0,0,0,0,0,"-87.69827696299996,41.85521453400003"
4700,-73.944228,40.654842,UNIVERSITY HOSPITAL OF BROOKLYN,BROOKLYN,NY,342,-999,342,POLYGON ((-73.93890275579102 40.65717873926209...,POLYGON ((-73.93890275579102 40.65717873926209...,1747103.0,0,0,0,0,0,"-73.94422785399995,40.654841879000045"
2863,-80.210637,25.776846,SELECT SPECIALTY HOSPITAL-MIAMI,MIAMI,FL,47,-999,47,POLYGON ((-80.16783914480926 25.78075315430153...,POLYGON ((-80.16783914480926 25.78075315430153...,1319063.0,0,0,0,0,0,"-80.21063654299998,25.77684554400003"
2994,-98.490984,29.441803,METROPOLITAN METHODIST HOSPITAL,SAN ANTONIO,TX,354,-999,354,POLYGON ((-98.50771390680109 29.47945010555169...,POLYGON ((-98.50771390680109 29.47945010555169...,1195461.0,0,0,0,0,0,"-98.49098396799995,29.441803431000036"


Why the analysis is flawed: hospitals don't have explicit coverage zones, it's possible a hospital's coverage zone contains no actual zip codes, thus no patients would be allocated there. Similarly, a hospital located downtown in a major city will have a total_population that is overestimated, since people living there could go to hospitals that are farther away.

Solution: treat zip codes as distributions of populations. 

In [32]:
# a is the index
import numpy as np
from scipy.stats import expon
from scipy.spatial import distance_matrix

# mean distance between two regions
distances_mat = distance_matrix(p_df[['X','Y']].values, p_df[['X','Y']].values)
mean_distance = np.mean(distances_mat)

In [33]:
mean_log_pop = np.mean(np.log(p_df['total_population'] + 1))
print(mean_log_pop)

scale = mean_log_pop / mean_distance
print(scale)

7.985957442345193
0.4328525895471698


In [122]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [161]:
def p_new_infection_in_a_from_b(a, r0 = 2.2, duration = 14, sim = False):
    
    # sum of b over B
    # (n_a / d(a,b)) *  (c_b,t / n_b) p(communicate)
    n = sigmoid(np.log(p_df['total_population']) - np.log(p_df['total_population'][a]))
    distances = distances_mat[a]
    
    c_b = p_df[['asymptomatic_cases', 'critical_cases', 'severe_cases', 'mild_cases']].sum(axis=1)
    if sim:
        c_b = np.random.exponential(np.exp(mean_log_pop), size=(len(p_df))).round()
    
    p_communicate = r0 / duration
    travel = distances / np.exp(n) # here, close distances or large populations indicate higher chance of travel 
    p_travel = expon.pdf(travel, scale = scale) # scale is the avg score, loc is to keep the numbers realistic
    
    potential_infections_from_b = np.nan_to_num((p_travel * c_b)).round().astype('int64')
    
    communicated_cases = np.random.binomial(potential_infections_from_b, p_communicate)
    
    return communicated_cases


In [162]:
p_new_infection_in_a_from_b(0, sim=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log


array([643, 233, 190, ...,   0,   0,   0])

In [163]:
import numpy as np

t_df = pd.DataFrame(np.zeros((len(p_df['NAME']), 365)), index = p_df['NAME'])
t_df.head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
NAME,,,,,,,,,,,,,,,,,,,,,
LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CAMPUS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EAST LOS ANGELES DOCTORS HOSPITAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KINDRED HOSPITAL BALDWIN PARK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LAKEWOOD REGIONAL MEDICAL CENTER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


What is the probability of recovery at the various phases? The total fatality is around 2.6%. We want to break this down by mild, severe and critical. I'm going to say that this pretty much is reflected through age [source](https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/). Thus

- mild fatality rate: 1%
- severe fatality rate: 4.9%
- critical fatality rate: 22.8%

That way the total fatality rate is composed of $.026 = .228(.05) + .049(.14) + .01(.81)$.

The above logic is actually flawed, because the vast majority of people who die are the critical cases. Instead, we should think about the transition probabilities from one condition to the next.

Let's consider the five states of disease: asymptomatic, mild, severe, critical, and dead. We know that the fatality in total is .023, and that .81 cases are mild, .14 cases are critical, and .05 cases are critical. Assuming we don't skip over states, we can say that
$$
p_1 = .81,
$$
$$
p_1 p_2 = .14,
$$
$$
p_1 p_2 p_3 = .05,
$$
and 
$$
p_1 p_2 p_3 p_4 = .023.
$$

Solving for the state transitions yields $p_1 = .81, p_2 = .1728, p_3 = .357, p_4 = .46$.

In [164]:
# iterator
def iterator():
    # calculate all the new cases
    asymptomatic_cases = [np.sum(p_new_infection_in_a_from_b(i, sim=True)) for i in range(len(p_df))]
    
    asymp_to_mild = asymptomatic_cases * 
    
    print(new_cases)

In [165]:
iterator()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in greater_equal
  return (a <= x) & (x <= b)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in less_equal
  return (a <= x) & (x <= b)


[76941, 154222, 141803, 176417, 142436, 138509, 158247, 155297, 107032, 15724, 34406, 147418, 114277, 119749, 125320, 28269, 36081, 35251, 36427, 34613, 50504, 53101, 39501, 145347, 82008, 141961, 160847, 69796, 23746, 39583, 52179, 31044, 91460, 47936, 27817, 94980, 45327, 33329, 35743, 50948, 104352, 94135, 96684, 66500, 84758, 59749, 67938, 72978, 76821, 68722, 79291, 76071, 81574, 52070, 112534, 101481, 62020, 49364, 107698, 137963, 119540, 169443, 100288, 102963, 121785, 135413, 41305, 32489, 35256, 58383, 67435, 68328, 86230, 63601, 49725, 44338, 20970, 133501, 74163, 46114, 54867, 75958, 51706, 60456, 19288, 49794, 45553, 54261, 22714, 74279, 118495, 107726, 94664, 119505, 98552, 33076, 20398, 99461, 39064, 32670, 73945, 25214, 19472, 30174, 30525, 29344, 116491, 127962, 43170, 130058, 28329, 144098, 62570, 57422, 58869, 79160, 111029, 29832, 63657, 41497, 41777, 119428, 92131, 42239, 13009, 7340, 26635, 30953, 73988, 68389, 88311, 86646, 91316, 43706, 89610, 69079, 66136, 48216

A way to validate our simulation is that we know that the number of cases grows about 1.155x each day in aggregate.

## Visualizing

In [1]:
# plotting 

PATH = 'cb_2018_us_county_within_cd116_500k.shx'
usa = gpd.read_file(PATH)

fig = plt.figure(figsize=(100,100))
ax1 = fig.add_subplot(111)

voronoi_plot_2d(h_vor, ax = ax1, show_vertices=False)

usa.plot(ax = ax1, color = 'grey')
z_gdf.plot(ax = ax1, markersize = 1)

NameError: name 'gpd' is not defined

In [ ]:
fig.savefig("plot.png")

## Bibliography

- [coronavirus numbers](https://ourworldindata.org/coronavirus)